In [22]:
import os, sys, pickle, random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.cm as cm
from tqdm import tqdm
from pathlib import Path
import mne
if os.getcwd().split("/")[-1] != 'BENDR-XAI': os.chdir("../")
from notebooks.utils import *

ModuleNotFoundError: No module named 'torch'

In [2]:
data = np.load("data/HCPMMP1_combined_8.0_12.0_220503_280423.npy", allow_pickle=True).item()

In [3]:
count = 0
for name in data.keys():
    count += len(data[name]['annotations']['T0'])
count / 148

In [39]:
minimum = -0.00125
maximum = 0.00125
(x - minimum) / (maximum - minimum) * 2 - 1

In [89]:
# We use T3 and T4 instead of T7 and T8
channels_order = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']

data = np.load("data/HCPMMP1_combined_8.0_12.0_220503_280423.npy", allow_pickle=True).item()

for edf_file, result_dict in tqdm(data.items(), total=len(data)):
    raw = read_TUH_edf(f"/scratch/s194260/tuh_eeg/{edf_file}", low_pass=None)
    try:
        raw = raw.reorder_channels(channels_order)
    except ValueError:
        continue
    
    new = (raw.get_data() - minimum) / (maximum - minimum) * 2 - 1
    print(new.min(), new.max())

    #for anno in tqdm(result_dict['annotations']['T0']):
    #    window = get_window(raw, anno).get_data()
    #    minimum = np.minimum(window.min(axis=1), minimum)
    #    maximum = np.maximum(window.max(axis=1), maximum)
        
#minimum = minimum.reshape(-1, 1)
#maximum = maximum.reshape(-1, 1)

  1%|          | 2/200 [00:00<00:36,  5.41it/s]

-0.5097346304042139 0.36637263968230815
-0.5286213530505284 0.3576066638852662


  2%|▏         | 4/200 [00:00<00:36,  5.42it/s]

-0.39743511610083904 0.36127378944182986
-0.43169957821461113 0.3595168264756474


  2%|▎         | 5/200 [00:00<00:37,  5.23it/s]

...

-0.32724072536283744 0.3593176267757394


 35%|███▌      | 70/200 [00:28<01:32,  1.41it/s]

-0.3692268697706752 0.37852184975485925


 36%|███▌      | 72/200 [00:28<01:06,  1.92it/s]

-0.44378503900127486 0.37369800623427274
-0.36956278265439735 0.36548644005260056


In [2]:
# Make a list of path to all edf files in the subfolders of /nobackup/tsal-tmp/tuh_eeg
# files = []

# with tqdm() as pbar:
#     for root, dirs, file in os.walk("/nobackup/tsal-tmp/tuh_eeg"):
#         for name in file:
#             if name.endswith(".edf"):
#                 files.append(os.path.join(root, name))
#                 pbar.update(1)

# Save list of files as pickle
# import pickle
# with open('tuh_files.pkl', 'wb') as f:
#     pickle.dump(files, f)
    
# Load list of files from pickle
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)
    
# random.shuffle(files)

In [3]:
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)

# Copy all files in tuh_final_selected.txt to data/tuh_eeg/
# with open('tuh_final_selected.txt', 'r') as f:
#     selected = f.read().splitlines()

# selected = [file.split("/")[-1] for file in selected]

# Delete all files in /scratch/s194260/tuh_eeg/ that are not in selected
# for file in os.listdir('/scratch/s194260/tuh_eeg/'):
#     if file not in selected:
#         os.remove('/scratch/s194260/tuh_eeg/' + file)